In [2]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from copy import deepcopy

import transformers
from transformers import BertTokenizer

import seaborn
seaborn.set_style('whitegrid')



c:\Users\yarom\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\yarom\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\yarom\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
data = pd.read_excel('data_train.xlsx')
data

,posts,label
0,Как транcлитерировать украинские буквы на лати...,NaN
1,Лучшие озера Берлина: от популярных до скрытых...,NaN
2,Ограничения движения по Берлину на выходных\n\...,NaN
3,Одна из важнейших страховок в Германии — это R...,NaN
4,Люстгартен парк в Берлине\n\nПарк «Люстгартен»...,NaN
...,...,...
184,Три человека погибли в результате аварии на A2...,NaN
185,Предупреждение о плохой погоде в Берлине и Бра...,NaN
186,Тегельский праздник у гавани \n\n\n18 - 21 июл...,NaN
187,Погрузитесь в захватывающий мир адреналина на...,NaN


In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(np.array(data.posts), np.array(data.label), test_size=0.2)

In [8]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=False,
            return_tensors='pt',
            truncation=True
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [6]:
MAX_LEN = 256
BATCH_SIZE = 32

train_dataset = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, fc_out_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn1 = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, fc_out_size)
        self.rnn2 = nn.RNN(fc_out_size, hidden_size, num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids):
        x = self.embedding(input_ids)
        rnn_out1, _ = self.rnn1(x)
        out1 = self.fc1(rnn_out1[:, -1, :]).unsqueeze(1)
        rnn_out2, _ = self.rnn2(out1)
        out2 = self.fc2(rnn_out2[:, -1, :]).flatten()
        return self.sigmoid(out2)

embed_size = 256
hidden_size = 256
fc_out_size = 512
num_layers = 7
output_size = 1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = RNNModel(len(tokenizer), embed_size, hidden_size, fc_out_size, num_layers, output_size).to(device)

In [24]:
def train_model(model, train_loader, val_loader, device, num_epochs=30):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    best_f1 = -1000
    best_model_weights = deepcopy(model.state_dict())

    for epoch in tqdm(range(num_epochs), desc='Epoch Progress'):
        model.train()
        running_loss = 0.0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1} Training', leave=False):
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1} - Training loss: {epoch_loss}')

        model.eval()
        all_preds = []
        all_labels = []
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation', leave=False):
                input_ids = batch['input_ids'].to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids)
                preds = outputs.round().cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        epoch_val_loss = val_loss / len(val_loader)
        correct = 0
        for i in range(len(all_preds)):
            if all_preds[i] == all_labels[i]:
                correct += 1
        total = len(all_labels)

        f1_val = f1_score(all_labels, all_preds)
        print(f'Validation Loss: {epoch_val_loss}, Accuracy: {100 * correct / total}, F1: {f1_val}%')

        
        if f1_val >= best_f1:
            best_f1 = f1_val
            best_model_weights = deepcopy(model.state_dict())

    print('Finished Training')
    model.load_state_dict(best_model_weights)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
num_epochs = 25

train_model(model, train_loader, val_loader, device, num_epochs)

Epoch Progress:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1 Training:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1 - Training loss: 30.92833948135376


Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Validation Loss: 37.06378936767578, Accuracy: 65.625, F1: 0.6451612903225806%


...

Validation Loss: 35.85849380493164, Accuracy: 68.75, F1: 0.6666666666666666%


Epoch 25 Training:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 25 - Training loss: 28.940399646759033


Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Validation Loss: 35.85849380493164, Accuracy: 68.75, F1: 0.6666666666666666%
Finished Training


In [26]:
def preprocess_text(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=False,
        return_tensors='pt',
        truncation=True
    )
    return encoding['input_ids']


text = '''⚡️Вы живёте в Берлине , собираетесь туда переезжать или возможно просто путешествуете❓ 

Канал BERLIN LIVE  создан именно для вас!

✅ Ежедневные обновления о событиях в Берлине 

✅ Рассказы о знаменитых достопримечательностях и скрытых жемчужинах города

✅ Советы о том, где поесть, что посмотреть и как провести время в Берлине 

✅ Интересные истории о жизни в этом удивительном городе

👉 Ссылка на канал 
👉 Подписаться '''

input_id = preprocess_text(text).to(device)

with torch.no_grad():
    model.eval() 
    output = model(input_id)
    prediction_prob = output.item()

prediction = ''
if round(prediction_prob) == 1:
    prediction = 'Реклама'
else:
    prediction = 'Не реклама'


print(f"Prediction: {prediction}, probability: {float(str(prediction_prob)[:4])*100}%")

Prediction: Реклама, probability: 99.0%


In [11]:
# сейвить будем всю модель

torch.save(model, 'model.pth')

# загрузка всей модели
# model = torch.load('model.pth')
# model.to(device)